In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model = ChatGroq(model="Gemma2-9b-It")

In [35]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    rating: str

In [36]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']
    
    # call llm generate outline
    prompt = f"Generate a detailed outline for a blog on the topic - {title}"
    outline = model.invoke(prompt).content
    
    # update state
    state['outline'] = outline

    return state


def create_blog(state: BlogState) -> BlogState:

    # fetch title and outline
    title = state['title']
    outline = state['outline']
    
    # call llm generate blog
    prompt = f"Write a detailed blog on the title - {title} \n\n using the following outline:\n {outline}"
    content = model.invoke(prompt).content
    
    # update state
    state['content'] = content

    return state


def rate_blog(state: BlogState) -> BlogState:
    outline = state['outline']
    blog = state['content']

    prompt = f"Based on this outline: {outline} \n. Rate my blog out of 10: {blog} \n\n. Give the rating number only based on how much my Blog aligns with the outline"
    response = model.invoke(prompt).content

    state['rating'] = response

    return state    

In [37]:
graph = StateGraph(BlogState)

# nodes
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)
graph.add_node("rate_blog",rate_blog)


# edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',"rate_blog")
graph.add_edge("rate_blog", END)


#compile

workflow = graph.compile()

In [46]:
initial_state = {'title': "Rise of AI in Computer Vision"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in Computer Vision', 'outline': "## Rise of AI in Computer Vision: A Detailed Blog Outline\n\n**I. Introduction**\n\n*  Hook: Start with a captivating example of AI's impact on computer vision (e.g., self-driving cars, medical imaging analysis).\n*  Define computer vision: Briefly explain what computer vision is and its significance.\n*  Introduction to AI: Touch upon the different types of AI and how they contribute to computer vision advancements.\n*  Thesis statement: State the blog's main argument - AI is revolutionizing computer vision, leading to groundbreaking applications and transforming various industries.\n\n**II. Historical Context**\n\n*  Early days of computer vision: Briefly discuss the history of computer vision, highlighting key milestones and limitations.\n*  The role of traditional methods: Explain how traditional computer vision techniques relied heavily on manual feature engineering and rule-based systems.\n*  The AI revolution: Discuss the em

In [47]:
print(final_state['outline'])

## Rise of AI in Computer Vision: A Detailed Blog Outline

**I. Introduction**

*  Hook: Start with a captivating example of AI's impact on computer vision (e.g., self-driving cars, medical imaging analysis).
*  Define computer vision: Briefly explain what computer vision is and its significance.
*  Introduction to AI: Touch upon the different types of AI and how they contribute to computer vision advancements.
*  Thesis statement: State the blog's main argument - AI is revolutionizing computer vision, leading to groundbreaking applications and transforming various industries.

**II. Historical Context**

*  Early days of computer vision: Briefly discuss the history of computer vision, highlighting key milestones and limitations.
*  The role of traditional methods: Explain how traditional computer vision techniques relied heavily on manual feature engineering and rule-based systems.
*  The AI revolution: Discuss the emergence of deep learning and its transformative impact on computer v

In [48]:
print(final_state['content'])

## Rise of AI in Computer Vision: A Detailed Blog 

**I. Introduction**

Imagine a world where computers can "see" and understand images just like humans do.  This isn't science fiction anymore; it's the reality we're rapidly approaching thanks to the transformative power of artificial intelligence (AI) in computer vision.  

Computer vision is the field of AI that enables computers to interpret and analyze visual information from the real world.  It's used in a wide range of applications, from self-driving cars to medical diagnosis, and its impact is already being felt across industries.  

This blog explores the exciting rise of AI in computer vision, delving into its history, key techniques, groundbreaking applications, and the ethical considerations that come with this powerful technology.

**II. Historical Context**

The quest to give computers "sight" dates back decades. Early computer vision systems relied heavily on manual feature engineering, where programmers painstakingly de

In [49]:
print(final_state['rating'])

9 

